In [3]:
from datetime import datetime
import re, json, pandas as pd, numpy as np, os
import pdfplumber

pdf_dir = "data_pdfs"
out_dir = "data"
os.makedirs(out_dir, exist_ok=True)


from datetime import datetime

pos_cols = [
    "prod_long","prod_short",
    "swap_long","swap_short","swap_spreading",
    "mm_long","mm_short","mm_spreading",
    "other_long","other_short","other_spreading"
]

largest_cols = [
    "gross_4_long","gross_4_short","gross_8_long","gross_8_short",
    "net_4_long","net_4_short","net_8_long","net_8_short"
]

commodity_line_pat = re.compile(r"^([A-Z0-9][A-Z0-9 .,&'\/\-]+?)\s-\s([A-Z0-9][A-Z0-9 .,&'\/\-]+?)\s*$")
date_line_pat = re.compile(r"^Disaggregated Commitments of Traders - Futures Only,\s*(.+?)\s*$")
contracts_line_pat = re.compile(r"^\s*:\s*:\(CONTRACTS? OF (.+?)\)\s*$")

positions_row_pat = re.compile(r"^(All|Old|Other)\s*:\s*([0-9,]+)\s*:\s*([0-9,.\- ]+?)\s*$")
change_row_pat = re.compile(r"^\s*:\s*([0-9,\-]+)\s*:\s*([0-9,.\- ]+?)\s*$")
largest_row_pat = re.compile(r"^(All|Old|Other)\s*:\s*([0-9.\- ]+?)\s*$")

def parse_int(s):
    s = s.strip().replace(",", "")
    if s in {".", ""}:
        return None
    try:
        return int(s)
    except:
        return None

def parse_float(s):
    s = s.strip().replace(",", "")
    if s in {".", ""}:
        return None
    try:
        return float(s)
    except:
        return None

def parse_11_numbers(blob, floaty=False):
    parts = [p for p in re.split(r"\s+", blob.strip()) if p][:11]
    vals = [parse_float(p) if floaty else parse_int(p) for p in parts]
    vals += [None] * (11 - len(vals))
    return dict(zip(pos_cols, vals))

def parse_8_numbers(blob):
    parts = [p for p in re.split(r"\s+", blob.strip()) if p][:8]
    vals = [parse_float(p) for p in parts]
    vals += [None] * (8 - len(vals))
    return dict(zip(largest_cols, vals))

def ensure_struct(comm, exch, date, contract):
    dataset.setdefault(comm, {"exchange": exch, "report_date": date, "contract_spec": contract, "rows": {}, "change": {}})
    for sr in ["All","Old","Other"]:
        dataset[comm]["rows"].setdefault(sr, {})

def date_from_filename(fname):
    # accept either DD:MM:YY or DD/MM/YY (just in case)
    m = re.search(r"(\d{2})[:/](\d{2})[:/](\d{2})", fname)
    if not m:
        raise ValueError(f"No date found in filename: {fname}")

    d = datetime.strptime(m.group(0), "%d:%m:%y") if ":" in m.group(0) else datetime.strptime(m.group(0), "%d/%m/%y")
    return d.strftime("%Y-%m-%d")

for pdf_file in sorted(f for f in os.listdir(pdf_dir) if f.lower().endswith(".pdf")):
    pdf_path = os.path.join(pdf_dir, pdf_file)
    report_date_tag = date_from_filename(pdf_file)

    # ---------- RESET STATE PER PDF ----------
    all_lines = []
    dataset = {}
    current = None
    current_exchange = None
    current_report_date = None
    current_contract_size = None
    state = None
    pending_change = False
    # ----------------------------------------

    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            txt = page.extract_text() or ""
            for line in txt.splitlines():
                all_lines.append(line.rstrip())

    for line in all_lines:
        m = commodity_line_pat.match(line)
        if m:
            current = m.group(1).strip()
            current_exchange = m.group(2).strip()
            current_contract_size = None
            state = None
            pending_change = False
            ensure_struct(current, current_exchange, current_report_date, current_contract_size)
            continue

        if current is None:
            continue

        m = date_line_pat.match(line)
        if m:
            current_report_date = m.group(1).strip()
            ensure_struct(current, current_exchange, current_report_date, current_contract_size)
            continue

        m = contracts_line_pat.match(line)
        if m:
            current_contract_size = m.group(1).strip()
            ensure_struct(current, current_exchange, current_report_date, current_contract_size)
            continue

        if "Positions" in line and line.strip().endswith("Positions"):
            state = "positions"
            continue
        if "Changes in Commitments from:" in line:
            pending_change = True
            continue
        if "Percent of Open Interest Represented by Each Category of Trader" in line:
            state = "percent"
            continue
        if "Number of Traders in Each Category" in line:
            state = "traders"
            continue
        if "Percent of Open Interest Held by the Indicated Number of the Largest Traders" in line:
            state = "largest"
            continue

        if pending_change:
            m = change_row_pat.match(line)
            if m:
                ensure_struct(current, current_exchange, current_report_date, current_contract_size)
                dataset[current]["change"]["open_interest"] = parse_int(m.group(1))
                dataset[current]["change"].update(parse_11_numbers(m.group(2), floaty=False))
                pending_change = False
            continue

        m = positions_row_pat.match(line)
        if m and state in {"positions","percent","traders"}:
            sr = m.group(1)
            open_interest = m.group(2)
            blob = m.group(3)
            ensure_struct(current, current_exchange, current_report_date, current_contract_size)
            if state == "positions":
                dataset[current]["rows"][sr]["open_interest"] = parse_int(open_interest)
                dataset[current]["rows"][sr].update(parse_11_numbers(blob, floaty=False))
            elif state == "percent":
                dataset[current]["rows"][sr].update({f"pct_{k}": v for k,v in parse_11_numbers(blob, floaty=True).items()})
            elif state == "traders":
                dataset[current]["rows"][sr].update({f"traders_{k}": v for k,v in parse_11_numbers(blob, floaty=False).items()})
            continue

        if state == "largest":
            m = largest_row_pat.match(line)
            if m:
                sr = m.group(1)
                blob = m.group(2)
                if re.search(r"[A-Za-z]", blob):
                    continue
                ensure_struct(current, current_exchange, current_report_date, current_contract_size)
                dataset[current]["rows"][sr].update({f"largest_{k}": v for k,v in parse_8_numbers(blob).items()})
                continue

    records = []
    for comm, info in dataset.items():
        for sr, row in info["rows"].items():
            rec = {
                "commodity": comm,
                "exchange": info.get("exchange"),
                "report_date": info.get("report_date"),
                "contract_spec": info.get("contract_spec"),
                "subreport": sr,
                "change_open_interest": info.get("change", {}).get("open_interest"),
            }
            for k in pos_cols:
                rec[f"change_{k}"] = info.get("change", {}).get(k)
            rec.update(row)
            records.append(rec)

    df = pd.DataFrame(records)

    desired = (
        ["commodity","exchange","report_date","contract_spec","subreport","open_interest"]
        + ["change_open_interest"] + [f"change_{k}" for k in pos_cols]
        + pos_cols
        + [f"pct_{k}" for k in pos_cols]
        + [f"traders_{k}" for k in pos_cols]
        + [f"largest_{k}" for k in largest_cols]
    )
    ordered = [c for c in desired if c in df.columns] + [c for c in df.columns if c not in desired]
    df = df[ordered]

    out_csv = os.path.join(out_dir, f"cot_ag_futures_{report_date_tag}_dataset.csv")
    df.to_csv(out_csv, index=False)

    print(f"Saved: {out_csv}")


Saved: data/cot_ag_futures_2025-06-24_dataset.csv
Saved: data/cot_ag_futures_2025-02-25_dataset.csv
Saved: data/cot_ag_futures_2025-03-25_dataset.csv
Saved: data/cot_ag_futures_2025-11-25_dataset.csv
Saved: data/cot_ag_futures_2025-08-26_dataset.csv
Saved: data/cot_ag_futures_2025-05-27_dataset.csv
Saved: data/cot_ag_futures_2025-01-28_dataset.csv
Saved: data/cot_ag_futures_2025-10-28_dataset.csv
Saved: data/cot_ag_futures_2025-04-29_dataset.csv
Saved: data/cot_ag_futures_2025-07-29_dataset.csv
Saved: data/cot_ag_futures_2025-09-30_dataset.csv
Saved: data/cot_ag_futures_2025-12-30_dataset.csv
